In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import pathlib
import os

In [2]:
BATCH_SIZE = 32
EPOCHS = 200
NUM_CLASSES = 10
SUBSTRACT_PIXEL_MEAN = False

N = 11
depth = N * 9 + 2

model_type = 'ResNet%d' % depth

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 14s 0us/step


In [4]:
input_shape = x_train.shape[1:]

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

if SUBSTRACT_PIXEL_MEAN:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [5]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [6]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

    x = inputs
    

In [7]:
def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = resnet_v2(input_shape=input_shape, depth=depth)

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])

Learning rate:  0.001


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch_normalization[0][0]']

In [9]:
save_dir = pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/MGU/Resnet')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [10]:
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

callbacks = [checkpoint, lr_scheduler]

In [ ]:
model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks
          )

Learning rate:  0.001
Epoch 1/200
1563/1563 [==============================] - ETA: 0s - loss: 2.3158 - accuracy: 0.5154

1563/1563 [==============================] - 145s 80ms/step - loss: 2.3158 - accuracy: 0.5154 - val_loss: 2.0323 - val_accuracy: 0.4775 - lr: 0.0010
Learning rate:  0.001
Epoch 2/200
1563/1563 [==============================] - ETA: 0s - loss: 1.4393 - accuracy: 0.6595

1563/1563 [==============================] - 116s 74ms/step - loss: 1.4393 - accuracy: 0.6595 - val_loss: 1.6356 - val_accuracy: 0.5802 - lr: 0.0010
Learning rate:  0.001
Epoch 3/200
1562/1563 [============================>.] - ETA: 0s - loss: 1.2072 - accuracy: 0.7170

1563/1563 [==============================] - 115s 73ms/step - loss: 1.2071 - accuracy: 0.7171 - val_loss: 1.2234 - val_accuracy: 0.7034 - lr: 0.0010
Learning rate:  0.001
Epoch 4/200
1563/1563 [==============================] - ETA: 0s - loss: 1.0627 - accuracy: 0.7574

1563/1563 [==============================] - 116s 74ms/step - loss: 1.0627 - accuracy: 0.7574 - val_loss: 1.4243 - val_accuracy: 0.6479 - lr: 0.0010
Learning rate:  0.001
Epoch 5/200
1563/1563 [==============================] - ETA: 0s - loss: 0.9722 - accuracy: 0.7827

...

1563/1563 [==============================] - 114s 73ms/step - loss: 0.0870 - accuracy: 1.0000 - val_loss: 0.8991 - val_accuracy: 0.8653 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 181/200
1562/1563 [============================>.] - ETA: 0s - loss: 0.0868 - accuracy: 1.0000

1563/1563 [==============================] - 115s 73ms/step - loss: 0.0868 - accuracy: 1.0000 - val_loss: 0.8906 - val_accuracy: 0.8660 - lr: 1.0000e-06
Learning rate:  5e-07
Epoch 182/200
1562/1563 [============================>.] - ETA: 0s - loss: 0.0868 - accuracy: 1.0000

1563/1563 [==============================] - 114s 73ms/step - loss: 0.0868 - accuracy: 1.0000 - val_loss: 0.8932 - val_accuracy: 0.8656 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 183/200
1563/1563 [==============================] - ETA: 0s - loss: 0.0869 - accuracy: 0.9999

1563/1563 [==============================] - 118s 76ms/step - loss: 0.0869 - accuracy: 0.9999 - val_loss: 0.8939 - val_accuracy: 0.8657 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 184/200
1562/1563 [============================>.] - ETA: 0s - loss: 0.0870 - accuracy: 0.9999

1563/1563 [==============================] - 115s 73ms/step - loss: 0.0870 - accuracy: 0.9999 - val_loss: 0.8936 - val_accuracy: 0.8654 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 185/200
1294/1563 [=======================>......] - ETA: 18s - loss: 0.0867 - accuracy: 1.0000

In [11]:
saved_model = pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/MGU/Resnet/cifar10_ResNet101_model.086.h5')
model2 = keras.models.load_model(saved_model)

In [21]:
cont_epoch = 87

def lr_schedule(epoch):
    lr = 1e-3
    if epoch + cont_epoch > 180:
        lr *= 0.5e-3
    elif epoch + cont_epoch > 160:
        lr *= 1e-3
    elif epoch + cont_epoch > 120:
        lr *= 1e-2
    elif epoch + cont_epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

callbacks = [checkpoint, lr_scheduler]

In [22]:
model2.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=EPOCHS - 86,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks
          )

Learning rate:  0.0001
Epoch 1/114
1563/1563 [==============================] - ETA: 0s - loss: 0.3466 - accuracy: 0.9782
Epoch 1: val_accuracy improved from 0.79330 to 0.87230, saving model to /content/drive/MyDrive/Colab_Notebooks/MGU/Resnet/cifar10_ResNet101_model.001.h5
1563/1563 [==============================] - 122s 78ms/step - loss: 0.3466 - accuracy: 0.9782 - val_loss: 0.7167 - val_accuracy: 0.8723 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 2/114
1562/1563 [============================>.] - ETA: 0s - loss: 0.3062 - accuracy: 0.9918
Epoch 2: val_accuracy improved from 0.87230 to 0.87690, saving model to /content/drive/MyDrive/Colab_Notebooks/MGU/Resnet/cifar10_ResNet101_model.002.h5
1563/1563 [==============================] - 121s 78ms/step - loss: 0.3062 - accuracy: 0.9918 - val_loss: 0.7202 - val_accuracy: 0.8769 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 3/114
1563/1563 [==============================] - ETA: 0s - loss: 0.2867 - accuracy: 0.9949
Epoch 3: val_accuracy 

In [24]:
model_directory = pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/MGU/ResNet101_CIFAR10.h5')

model2.save(model_directory)